# Getting Started with Querying the Database Using Python

## Load in the Database

In [ ]:
from astrodb_utils import load_astrodb
from simple import REFERENCE_TABLES

SCHEMA_PATH = "../../simple/schema.yaml" 
recreatedb = False #Set to True or False
db = load_astrodb(
     "../../SIMPLE.sqlite", 
     recreatedb=recreatedb,  
     reference_tables=REFERENCE_TABLES, 
     felis_schema=SCHEMA_PATH
     )

## Query the Database

### Using Astrodbkit Query Functions

What is Astrodbkit? (Link: https://astrodbkit.readthedocs.io/en/latest/#exploring-the-schema)
AstrodbKit is an astronomical database handler code built on top of SQLAlchemy. We can implement various functions from the python package to query the database in robust ways.

In [2]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

Lets check the tables in the database

In [3]:
for table in db.metadata.tables:
    print(table)

CompanionList
CompanionParameters
Sources
Publications
CompanionRelationships
Gravities
Regimes
Instruments
Telescopes
ModeledParameters
Parameters
Names
Parallaxes
Photometry
PhotometryFilters
ProperMotions
RadialVelocities
RotationalParameters
Spectra
SpectralTypes
Versions


Now that we know the tables,lets do a more detailed search into the tables and their content

Lets look at the first 10 sources available within SIMPLE

In [4]:
db.query(db.Sources).limit(10).table()

source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str24,float64,float64,object,object,object,str11,object,object
2MASS J03552014+1439297,58.83375,14.658056,None,None,None,Cruz07,None,None
1RXS J115928.5-524717,179.864,-52.7886,None,2000,None,Hamb04.265,None,None
ULAS J114925.58-014343.2,177.3569,-1.7287,None,2000,None,Burn10.1885,None,None
DENIS-P J1756561-480509,269.2342,-48.086,None,2000,None,Phan08,None,None
PSO J024.4369+09.1360,24.4372,9.1366,None,2000,None,Skrz16,None,Best20 lists discovery as Skrz16; Best20a
SDSS J161420.50+004643.6,243.585,0.7785,None,2000,None,Hawl02,None,None
SDSS J094903.15+264944.2,147.2633,26.8288,None,2000,None,Schm10.1808,None,None
SDSS J083717.21-000018.0,129.3215835,-0.004972,None,None,0837-0000,Legg00,None,None
2MASS J06080232-2944590,92.0097,-29.7497,None,2000,None,Cruz03,None,None


Thats a lot of sources! Lets say you just want to find all sources that start with "WISE" 
We can use the search_object function for this
Search_Object documentation: https://astrodbkit.readthedocs.io/en/latest/#identifier-name-search

In [5]:
db.search_object('wise', fmt='astropy')

Using table 'Sources' with columns ['source'] for matching object names
Using table 'Names' with columns ['other_name'] for matching object names


source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str32,float64,float64,object,object,object,str11,object,object
1RXS J115928.5-524717,179.864,-52.7886,None,2000,None,Hamb04.265,None,None
2MASS J00011217+1535355,0.300708,15.593194,None,None,0001+1535,Knap04,None,None
2MASS J00034227-2822410,0.926125,-28.378056,None,None,0003-2822,Cruz07,None,None
2MASS J00040288-6410358,1.012,-64.176611,None,None,0004-6410,Kirk10,None,None
2MASS J00043484-4044058,1.145166,-40.734944,None,None,0004-4044BC,Eros99,None,None
2MASS J00044144-2058298,1.1728,-20.975,None,2000,None,Kend07,None,None
2MASS J00045753-1709369,1.2398,-17.1603,None,2000,None,Crif05,None,None
2MASS J00054844-2157196,1.4519,-21.9555,None,2000,None,Reyl04,None,None
2MASS J00070787-2458042,1.7827917,-24.967833,None,None,0007-2458,Reyl04,None,None


### Search by Object

So you found the object youre looking for - lets now narrow down that query.

Lets see many names are associated with the object WISEPC J0333

In [6]:
db.search_object('WISEPC J0333', resolve_simbad=True, output_table='Names')

No Simbad match for WISEPC J0333
Including Simbad names, searching for: ['WISEPC J0333']
Using table 'Sources' with columns ['source'] for matching object names
Using table 'Names' with columns ['other_name'] for matching object names


source,other_name
str26,str26
WISEPC J033349.34-585618.7,CWISE J033349.31-585619.8
WISEPC J033349.34-585618.7,VHS J033349.30-585620.2
WISEPC J033349.34-585618.7,WISEPC J033349.34-585618.7


### Inventory Check

We know the source exist, so now we want to see all the data associated with the source. 
Inventory documentation: https://astrodbkit.readthedocs.io/en/latest/#inventory-search

In [7]:
data = db.inventory('WISEPC J033349.34-585618.7', pretty_print=True)

{
    "Sources": [
        {
            "source": "WISEPC J033349.34-585618.7",
            "ra": 53.4557,
            "dec": -58.9384,
            "epoch": null,
            "equinox": "2000",
            "shortname": null,
            "reference": "Kirk11",
            "other_references": null,
            "comments": null
        }
    ],
    "Names": [
        {
            "other_name": "CWISE J033349.31-585619.8"
        },
        {
            "other_name": "VHS J033349.30-585620.2"
        },
        {
            "other_name": "WISEPC J033349.34-585618.7"
        }
    ],
    "Parallaxes": [
        {
            "parallax": 46.2,
            "parallax_error": 3.7,
            "adopted": true,
            "comments": null,
            "reference": "Kirk21"
        }
    ],
    "Photometry": [
        {
            "band": "IRAC.I1",
            "magnitude": 13.59,
            "magnitude_error": 0.02,
            "telescope": "Spitzer",
            "epoch": null,
            

### Search by String

Now lets do a broader search, say you want to find all objects with the publication reference "Cruz18" 

Within astrodbkit, we allow for the parameter format (fmt) so that you can decide how you wish to visualize your query. Your options are pandas, astropy, or default (default is an astropy table)

Search string documentation: https://astrodbkit.readthedocs.io/en/latest/#full-string-search

In [8]:
db.search_string('Cruz18', fuzzy_search=False, fmt='astropy')  #fmt options: pandas, astropy/table, default

Sources
         source               ra      ... other_references   comments  
----------------------- ------------- ... ---------------- ------------
2MASS J15382417-1953116     234.60071 ...             None         None
2MASS J00110940+5149236      2.789182 ...             None         None
2MASS J15063706+2759544     226.65421 ...             None       galaxy
2MASS J05583706-1339123     89.654445 ...             None         None
2MASS J23490528+1833150       357.272 ...             None         None
2MASS J23202927+4123415     350.12167 ...             None         None
2MASS J22433237-1525260    340.884859 ...             None         None
2MASS J12490458-3454080     192.26979 ...             None       galaxy
2MASS J23364145+4822480    354.172713 ...             None         None
2MASS J05574102-1333264     89.420954 ...             None        not M
                    ...           ... ...              ...          ...
2MASS J05574229-1333156     89.426209 ...             No

{'Sources': <Table length=24>
          source               ra      ... other_references   comments  
          str23             float64    ...      object         object   
 ----------------------- ------------- ... ---------------- ------------
 2MASS J15382417-1953116     234.60071 ...             None         None
 2MASS J00110940+5149236      2.789182 ...             None         None
 2MASS J15063706+2759544     226.65421 ...             None       galaxy
 2MASS J05583706-1339123     89.654445 ...             None         None
 2MASS J23490528+1833150       357.272 ...             None         None
 2MASS J23202927+4123415     350.12167 ...             None         None
 2MASS J22433237-1525260    340.884859 ...             None         None
 2MASS J12490458-3454080     192.26979 ...             None       galaxy
 2MASS J23364145+4822480    354.172713 ...             None         None
                     ...           ... ...              ...          ...
 2MASS J05574229-1333

### Search by Region 

Next example is query region, where you can use SkyCoord to create a coordinates object that query_region uses to query the database, in this example the Sources table. Make sure to use as accurate as a coordinate as possible. 

Query_Region Documentation: https://astrodbkit.readthedocs.io/en/latest/#region-spatial-search

Notes on Query Region: 
If you have the exact coordinates of the object you are searching you can use query_region as shown. However, if you don't you can enter the optional parameter "radius=x" to allow for a larger search radius, otherwise you will get back an empty query if your coordinates aren't precise enough

In [11]:
from astropy.units.quantity import Quantity

# An example of query with precise coordinates ----------
precise_coords = db.query_region(SkyCoord(144.395292, 29.528028, frame='icrs', unit='deg'), output_table='Sources', ra_col='ra', dec_col='dec')  
print(f'Table Results for (144.395292, 29.528028): \n {precise_coords}\n')

# An example that queries the database incorrectly
imprecise_coords = db.query_region(SkyCoord(144.3, 29.5, frame='icrs', unit='deg'), output_table='Sources', ra_col='ra', dec_col='dec') 
print(f'Table Results for (144.3, 29.5): \n {imprecise_coords}\n')

# An example that queries the database using the radius parameter
larger_search_area = db.query_region(SkyCoord(144.395, 29.528, frame='icrs', unit='deg'), radius=Quantity(60., unit='arcsec'), output_table='Sources', ra_col='ra', dec_col='dec',)
print(f'Table Results for (144.395, 29.528, radius=60.0 arcsecs): \n {(larger_search_area)}\n')

Table Results for (144.395292, 29.528028): 
          source             ra        dec    ... other_references comments
----------------------- ---------- --------- ... ---------------- --------
2MASS J09373487+2931409 144.395292 29.528028 ...             None     None

Table Results for (144.3, 29.5): 
 <No columns>

Table Results for (144.395, 29.528, radius=60.0 arcsecs): 
          source             ra        dec    ... other_references comments
----------------------- ---------- --------- ... ---------------- --------
2MASS J09373487+2931409 144.395292 29.528028 ...             None     None

